In [ ]:
import pandas as pd
import numpy as np
# from sklearn.daatsets import fetch_california_housing
# housing=fetch_california_housing
# housing.head()

### Take a look at the dataset of the california

In [ ]:
cf=pd.read_csv('housing.csv')
cf.head()

# Notes :- 
## 1. Each row or record represents one district and each records having 10 attributes.

In [ ]:
cf.info()

## By running the command of the info() total no. of record of each attributes datatypes with counting of null value shown

## for chacking the null values of the dataset write

In [ ]:
cf.isnull().any()

In [ ]:
cf.isnull().sum().sum()

In [ ]:
cf['total_bedrooms'].isnull().sum()

## The next observation the given attributes are in the numeric datatype except the ocean_proximity 

In [ ]:
cf['ocean_proximity'].value_counts()

## ocean_proximity is categories which say that how many of the each district belong to which category.

In [ ]:
cf.describe()

## 1. The Count , mean , min and max rows are self explainatory , the null values are ignored so for example the count of total_bedrooms is 20433 not 20604 the std row shows the standard deviation which measures how dispersed the value are,
## 2. The 25% , 50% and 75 % rows shows the corresponding percentile, a percentile indicates the value below which a given percentage of observation in a group of observation fail. 


## Let's plotting the histogram to shows the number of instances on the vertical axis that have a gievn value range (on horizontal axis) you can either plot this one attributes at a time or you can call the hist method on the whole dataset.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
cf.hist(bins=90,figsize=(20,15),legend=True,grid=True) # gird is used for grid (boxes) in graph
plt.savefig('Histogram_Plotting_california')
plt.legend()
plt.show()

## Median income attribute does not look like it direct number it is scalled between the range of minimum at 0.5 and called at 15.1 the number represent roughly ten of thousand of doller example where 3 is representing 30,000 doller.

## The housing median age and median house value were also capped.

In [ ]:
cf.head(2)

In [ ]:
a=np.random.seed(42)
print(a)


# Split training and testing data by random permutation

In [ ]:
def split_train_test(data,test_ratio):
#     shuffled_indicas=np.random.seed(42)
    shuffled_indices=np.random.permutation(len(data))
    test_set_size=int(len(data)*test_ratio)
    test_indices=shuffled_indices[:test_set_size]
    train_indices=shuffled_indices[test_set_size:]
    return data.iloc[train_indices],data.iloc[test_indices]

In [ ]:
shuffled_indices=np.random.permutation(len(cf))
print(shuffled_indices)

In [ ]:
train_set,test_set=split_train_test(cf,0.2)

In [ ]:
print(len(train_set))
print(len(test_set))

# Split training and testing data by sklearn.model 

In [ ]:
from sklearn.model_selection import train_test_split
train_set,test_set=train_test_split(cf,test_size=0.2,random_state=42)
print(len(train_set))
print(len(test_set))

In [ ]:
cf.head(3)

# Split training and testing data with indexes

In [ ]:
from zlib import crc32
def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio *2**32

In [ ]:
def split_train_test_by_id(data,test_ratio,id_column):
    ids=data[id_column]
    in_test_set=ids.apply(lambda id_:test_set_check(id_,test_ratio))
    return data.loc[~in_test_set],data.loc[in_test_set]


In [ ]:
cf_with_id=cf.reset_index()
train_set,test_set=split_train_test_by_id(cf_with_id,0.2,'index')

In [ ]:
print(train_set.shape)

In [ ]:
train_set.head()

In [ ]:
train_set.keys()

## For getting more stable unique identifier just fix with district's latitude longitude

In [ ]:
cf_with_id['id']=cf['longitude']*1000+cf['latitude']
train_set,test_set=split_train_test_by_id(cf_with_id,0.2,'id')

In [ ]:
train_set.shape

In [ ]:
train_set.head()

In [ ]:
train_set.keys()

In [ ]:
cf['median_income'].describe()

## Median income histogram more closely, most of the median income value clustered arround 1.5 to 6 doller that 15000 to 60000 doller but some median income go far beyond the 6 next the code to create an income category by using pd.cut function to create.

In [ ]:
cf['median_income'].hist()

## numpy module cann also representing value negative and positive.

In [ ]:
cf['income_cat']=pd.cut(cf['median_income'],bins=[0.,1.5,3.0,4.5,6.,np.inf],
                       labels=[1,2,3,4,5])

In [ ]:
cf['income_cat'].value_counts()

In [ ]:
cf.head()

In [ ]:
cf['income_cat'].hist()

# Date : 21-06-2023

## in statistic , stratified sampling is a method of sampling from a population which can be partitioned into subpopulation. in statistical survey when subpopulation within a overall population very it could be advantegous to sample each subpopulation independently.

## let's do the stratified sampling based on the income category for this you can use scikit learn stratifiedshuffleslip class

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
split=StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index,test_index in split.split(cf,cf['income_cat']):
    strat_train_set=cf.loc[train_index]
    strat_test_set=cf.loc[test_index]

In [ ]:
strat_test_set['income_cat'].value_counts()/len(strat_test_set)

## <font color='red'> now let's measure the income category proportions in the full data-set, and also compare the income category proportion in the overall dataset, in the test set generated with stratified sampling . and in a test set generated using purely random sampling.


## <font color='green'> As you can see the test set generated using startified sampling has income category proportion almost identical.

In [ ]:
def income_cat_proportions(data):
    return data['income_cat'].value_counts()/len(data)
train_set,test_set=train_test_split(cf,test_size=0.2,random_state=42)

In [ ]:
#compare the proportion
compare_props=pd.DataFrame({
    "Overall":income_cat_proportions(cf),
    "Stratified":income_cat_proportions(strat_test_set),
    "Random":income_cat_proportions(test_set)
}).sort_index()
compare_props['Rand. %Error']=100*compare_props['Random']/compare_props['Overall']-100
compare_props['Start. %Error']=100*compare_props['Stratified']/compare_props['Overall']-100

In [ ]:
print(compare_props)

In [ ]:
compare_props

In [ ]:
strat_test_set.head()

## now you should remove the income_cat attributes so the data is back to its original state.

In [ ]:
for set_ in(strat_train_set,strat_test_set):
    set_.drop("income_cat",axis=1,inplace=True)

In [ ]:
strat_train_set.head()

In [ ]:
cf=strat_train_set.copy()

In [ ]:
cf.head()

In [ ]:
#for the geographical visualization of data we have
#Longitude and latitude features and lets plots
#this feature in scatter plotting
cf.plot(kind='scatter',x='longitude',y='latitude')
plt.xlabel('longitude geographical')
plt.ylabel('latitude geographical')
plt.show('plotting of latitude and longitude')
plt.show()

## the above geographical visualization is much looking like a california population description but for observation the high density for the population setting the alpha =0.1 or for checking the high density of the data point.

In [ ]:
cf.plot(kind='scatter',x='longitude',y='latitude',alpha=0.1)
plt.xlabel('longitude geographical')
plt.ylabel('latitude geographical')
plt.show('plotting of latitude and longitude')
plt.show()

## Now that's you can see clearly see the high density areaa namely the bay area and around los angleles and san diengo.

In [ ]:
cf.head()

In [ ]:
cf.plot(kind='scatter',x='longitude',y='latitude',alpha=0.4,s=cf['population']/100,
        label='population',figsize=(10,4),c='median_house_value',
       cmap=plt.get_cmap('jet'),colorbar=True,sharex=True)
plt.xlabel('longitude geographical')
plt.ylabel('latitude geographical')
plt.title('plotting of latitude and longitude')
plt.show()

In [ ]:
import matplotlib.image as mpimg
cl=mpimg.imread('cal.jpg')
plt.imshow(cl)
plt.show()

In [ ]:
ax=cf.plot(kind='scatter',x='longitude',y='latitude',alpha=0.4,s=cf['population']/100,
        label='population',figsize=(15,6),c='median_house_value',
       cmap=plt.get_cmap('jet'),colorbar=True,sharex=True)
plt.imshow(cl,extent=[-124.55,-113.80,32.45,42.05],alpha=0.5,cmap=plt.get_cmap('jet'))
plt.xlabel("Longitude Geographical")
plt.ylabel("Latitude Geographical")
plt.title("Plotting of latitude and longitude")

prices=cf['median_house_value']
ticks_values=np.linspace(prices.min(),prices.max(),11)
cbar=plt.colorbar(ticks=ticks_values/prices.max())
cbar.ax.set_yticklabels(["$%dk"%(round(v/1000))for v in ticks_values],fontsize=14)
cbar.set_label('median house value',fontsize=16)
plt.legend(fontsize=12)
plt.savefig("california")
plt.show()

## since the dataset is not too large so lets do the standard correlation coefficient(also called pearson's) between every pair of attributes using the corr()

In [ ]:
import warnings
warnings.filterwarnings('ignore')
corr_matrix=cf.corr()
corr_matrix

In [ ]:
import seaborn as sns
fig=plt.figure(figsize=(12,8))
fig.add_subplot(1,1,1)
sns.heatmap(cf.corr(),annot=True,cmap='Oranges')

## Let's looks at how such each attributes correlated with the median house value

In [ ]:
corr_matrix['median_house_value'].sort_values(ascending=False)

## The correlation coefficient ranges from -1 to 1 when it is close to attribute, it means that there is a strong positive correlation, for example the median house value tends to go up. when the coeffiecient is close to -1 , it means that there is a strong negative correlations you vcan see a small negative correlation between the latitude and the median house value , finally the coeffcient close to 0 means that there is not linear correlation.

## Another way to check for the correlation between attributes is to use the pandas scatter_matrix function, which plot every numerical attributes againts every other numerical attributes , since there is 11 numerical attributes you would used 11^2 =121 which would not fit.

## <font color='green'>So, let's just focus on a few promising attributes that seems most correlated with the median housing value.

In [ ]:
from pandas.plotting import scatter_matrix
attributes=['median_house_value','median_income','total_rooms']
scatter_matrix(cf[attributes],figsize=(12,8))
plt.tight_layout()
plt.show()

# This scatter plotting visualize the numerical attributes against every other numerical attributes, plus histogram of each numerical attributes.
 ## The main diagonal (top left to bottom right) would be full of straight line if pandas plotted each variance against itself, which would not be very use full.
 ## The most promosing attributes to predict the median house value is the median income, so lets zoom in on their correlation scatterplot.

In [ ]:
cf.plot(kind='scatter',x='median_income',y='median_house_value',alpha=0.1)
plt.xlabel('median_income')
plt.ylabel('median_house_value')
plt.title('median income versus median house value')
plt.show()

## This plot reveals a few things, first the correlation is indeed very strong; you can clearly see the upward trend and the point are not too dispersed.

## Second the price cap that we noiced earlier is clearly visible as a horizonal line around doller 500000, but this plot reveals other less staight line horizontal line around 280000 and few more below that.

In [ ]:
cf.info()

In [ ]:
cf['rooms_per_household']=cf['total_bedrooms']/cf['households']
cf['bedrooms_per_rooms']=cf['total_bedrooms']/cf['total_rooms']
cf['population_per_households']=cf['population']/cf['households']

In [ ]:
corr_matrix2=cf.corr()
corr_matrix2

In [ ]:
import seaborn as sns
fig=plt.figure(figsize=(12,8))
fig.add_subplot(1,1,1)
sns.heatmap(corr_matrix2,annot=True,cmap='Oranges')

## Prepare the data for machine learning

In [ ]:
x=strat_train_set.drop('median_house_value',axis=1)
y=strat_train_set['median_house_value'].copy()

In [ ]:
x.isnull().any()

In [ ]:
z=x.copy()

In [ ]:
z.isnull().any()

## Making dummy filling median value

In [ ]:
median=z['total_bedrooms'].median()
z['total_bedrooms'].fillna(median,inplace=True)

In [ ]:
z.isnull().any()

## next mathod 
### sklearn provide a handy class to take care of missing value,
### SimpleImputer is use to help missing values need to creat a instance attributes.

In [ ]:
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(strategy='median')

In [ ]:
housing_num=x.drop('ocean_proximity',axis=1)

In [ ]:
imputer.fit(housing_num)

In [ ]:
imputer.statistics_

In [ ]:
X=imputer.transform(housing_num)

In [ ]:
X

In [ ]:
housing_tr=pd.DataFrame(X,columns=housing_num.columns,index=housing_num.index)

In [ ]:
housing_tr.head()

In [ ]:
x.head()

In [ ]:
housing_cat=x[['ocean_proximity']]
housing_cat.head(10)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
ordinal_encoder=OrdinalEncoder()
housing_cat_encoded=ordinal_encoder.fit_transform(housing_cat)

In [ ]:
ordinal_encoder.categories_

In [ ]:
housing_cat_encoded[:10]

# Date :- 22:06:2023

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
cat_encoder=OneHotEncoder()
housing_cat_hot1=cat_encoder.fit_transform(housing_cat)
housing_cat_hot1

In [ ]:
housing_cat_hot1.toarray()

In [ ]:
cat_encoder.categories_

# Transformation of pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
num_pipeline=Pipeline([
    ('imputer',SimpleImputer(strategy='median')),
    ('sts_scaler',StandardScaler())
])

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
num_attribs=list(housing_num)
cat_attribs=["ocean_proximity"]

In [ ]:
full_pipeline=ColumnTransformer([
    ("num",num_pipeline,num_attribs),
    ("cat",OneHotEncoder(),cat_attribs)
])

In [ ]:
housing_prepared=full_pipeline.fit_transform(x)

In [ ]:
x.head()

In [ ]:
x.isnull().any()

In [ ]:
housing_prepared

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
lin_reg=LinearRegression()
lin_reg.fit(housing_prepared,y)

In [ ]:
some_data=x.iloc[:5]
some_label=y.iloc[:5]
some_data_prepared=full_pipeline.transform(some_data)

In [ ]:
print("Prediction : ",lin_reg.predict(some_data_prepared))

In [ ]:
print('Labels',some_label)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
#Under fitting
housing_prediction=lin_reg.predict(housing_prepared)
lin_mse=mean_squared_error(y,housing_prediction)
lin_rmse=np.sqrt(lin_mse)
lin_rmse

In [ ]:
tree_reg=DecisionTreeRegressor()
tree_reg.fit(housing_prepared,y)

In [ ]:
# Overfitting
housing_prediction=tree_reg.predict(housing_prepared)
tree_mse=mean_squared_error(y,housing_prediction)
tree_rmse=np.sqrt(tree_mse)
tree_rmse

## cross validation :- 

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
scores=cross_val_score(tree_reg,housing_prepared,y,scoring='neg_mean_squared_error',cv=10)
tree_rmse_scores=np.sqrt(-scores)

In [ ]:
def display(scores):
    print("scores",scores)
    print("mean",scores.mean())
    print("Standard Deviation",scores.std())

In [ ]:
display(tree_rmse_scores)

In [ ]:
lin_scores=cross_val_score(lin_reg,housing_prepared,y,scoring='neg_mean_squared_error',cv=10)
lin_rmse_scores=np.sqrt(-lin_scores)

In [ ]:
display(lin_rmse_scores)

In [ ]:
forest_reg=RandomForestRegressor()
model=forest_reg.fit(housing_prepared,y)

In [ ]:
forest_reg.fit(housing_prepared,y)

In [ ]:
forest_scores=cross_val_score(forest_reg,housing_prepared,y,scoring='neg_mean_squared_error',cv=10)
forest_rmse_scores=np.sqrt(-forest_scores)

In [ ]:
display(forest_rmse_scores)

In [ ]:
import joblib

In [ ]:
joblib.dump(forest_reg,"my_cal_model.pkl")

## Fine_tune_model

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid=[{
    'n_estimators':[3,10,30],'max_features':[2,4,6,8]},
    {'bootstrap':[False],'n_estimators':[3,10],'max_features':[2,3,4]}
]

In [ ]:
forest_reg=RandomForestRegressor()
grid_search=GridSearchCV(forest_reg,param_grid,cv=5,scoring='neg_mean_squared_error',
                        return_train_score=True)
grid_search.fit(housing_prepared,y)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
cvres=grid_search.cv_results_

In [ ]:
for mean_score,param in zip(cvres["mean_test_score"],cvres["params"]):
    print(np.sqrt(-mean_score),param)

In [ ]:
final_model=grid_search.best_estimator_
X_test=strat_test_set.drop('median_house_value',axis=1)
Y_test=strat_test_set['median_house_value'].copy()
X_test_prepared=full_pipeline.transform(X_test)

In [ ]:
final_prediction=final_model.predict(X_test_prepared)

In [ ]:
final_mse=mean_squared_error(Y_test,final_prediction)

In [ ]:
final_rmse=np.sqrt(final_mse)

In [ ]:
final_rmse

In [ ]:
from scipy import stats

In [ ]:
confidence=0.95
squared_errors=(final_prediction-Y_test)**2
np.sqrt(stats.t.interval(confidence,len(squared_errors)-1,
                        loc=squared_errors.mean(),
                        scale=stats.sem(squared_errors)))

In [ ]:
import pickle

In [ ]:
with open('california_model','wb') as f:
    pickle.dump(final_model,f)

In [ ]:
from flask import Flask
from flask import request
from flask import render_template
import joblib
app = Flask(__name__)

model = joblib.load('california_model')
@app.route('/')
def hello(name=None):
    return render_template('index.html',name=name)


@app.route('/results', methods =["GET", "POST"])
def get_value():
    if request.method == "POST":
        log = float(request.form.get('lan'))
        lan = float(request.form.get('log'))
        mha = float(request.form.get('mha'))
        tr = float(request.form.get('tr'))
        tbr = float(request.form.get('tbr'))
        popula = float(request.form.get('popula'))
        hh = float(request.form.get('hh'))
        mi = float(request.form.get('mi'))
        mhv=float(request.form.get('mhv'))
        op = float(request.form.get('op'))
        predict_value = model.predict([[log,lan,mha,tr,tbr,popula,hh,mi,mhv,op,0,0,0]])
        print(predict_value)
    return render_template('results.html',prediction=[predict_value,lan,log])

In [ ]:
model

In [ ]:
if __name__=='__main__':
    app.run()